In [70]:
import os 
import sys
from sklearn.model_selection import train_test_split

from sklearn.utils import resample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier, Pool, cv

# from catboost_focal_loss import FocalLossObjective


%matplotlib inline

In [71]:
train = pd.read_csv("./ml.csv")
df_majority = train[train.label==0]
df_minority = train[train.label==1]

df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=39325,    # to match majority class
                                 random_state=123)
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
train_x = df_upsampled.drop('label', axis = 1)     # 删除表中的某一行或者某一列
train_y = df_upsampled['label']
print(train_x)

       feat  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  \
0         0           1           0           0           0           0   
1         0           1           0           0           0           0   
2         0           0           0           0           1           0   
3         1           0           0           0           0           0   
4         0           1           0           0           0           0   
...     ...         ...         ...         ...         ...         ...   
42063     1           0           0           0           0           0   
15693     0           0           1           0           0           0   
20174     0           0           0           0           0           1   
7824      1           0           0           0           0           0   
6986      0           0           1           0           0           0   

       Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  ...  Unnamed: 29  \
0               8      

In [72]:
Xy = train_test_split(train_x, train_y, test_size=0.2)
train_data, test_data, train_labels, test_labels = Xy

In [73]:
train_pool = Pool(train_data, train_labels)
test_pool = Pool(test_data, test_labels)

In [87]:
import math
from six.moves import xrange
from catboost import Pool, CatBoostClassifier
# from catboost_focal_loss import FocalLossObjective
params = {
    'loss_function' : 'Logloss',
    'custom_metric' : ['CrossEntropy', 'Accuracy'],
    'iterations' : 4000,        # 最大树数
    'learning_rate' : 0.1315,
    'l2_leaf_reg': 0.0345,        # L2正则化系数
    # 'colsample_bylevel': 0.5,     # 控制树的每一级的每一次分裂，对列数的采样的占比，跟rsm不共存
    'depth' : 16,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bayesian',
    'rsm' : 0.5927,     #  随机子空间
    'min_data_in_leaf': 9,
    'random_strength': 13.196,      # 分数标准差乘数
    'one_hot_max_size': 9,
    'bagging_temperature': 6.6325
}
#
# class FocalLossObjective(object):
#     def calc_ders_range(self, approxes, targets, weights):
#         # approxes, targets, weights are indexed containers of floats
#         # (containers with only __len__ and __getitem__ defined).
#         # weights parameter can be None.
#         # Returns list of pairs (der1, der2)
#         gamma = 2.
#         # alpha = 1.
#         assert len(approxes) == len(targets)
#         if weights is not None:
#             assert len(weights) == len(approxes)
#
#         exponents = []
#         for index in xrange(len(approxes)):
#             exponents.append(math.exp(approxes[index]))
#
#         result = []
#         for index in xrange(len(targets)):
#             p = exponents[index] / (1 + exponents[index])
#
#             if targets[index] > 0.0:
#                 der1 = -((1-p)**(gamma-1))*(gamma * math.log(p) * p + p - 1)/p
#                 der2 = gamma*((1-p)**gamma)*((gamma*p-1)*math.log(p)+2*(p-1))
#             else:
#                 der1 = (p**(gamma-1)) * (gamma * math.log(1 - p) - p)/(1 - p)
#                 der2 = p**(gamma-2)*((p*(2*gamma*(p-1)-p))/(p-1)**2 + (gamma-1)*gamma*math.log(1 - p))
#
#             if weights is not None:
#                 der1 *= weights[index]
#                 der2 *= weights[index]
#
#             result.append((der1, der2))
#
#         return result

In [88]:
clf = CatBoostClassifier(**params, early_stopping_rounds=100, eval_metric="Logloss")

In [89]:
clf.fit(train_pool, plot=True, eval_set=test_pool)
preds_raw = clf.predict(test_data,
                            prediction_type='RawFormulaVal')
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_pred = clf.predict(test_data)
print('AUC→',roc_auc_score(test_labels, y_pred))
print('ACC→',accuracy_score(test_labels, y_pred))
print('F1 score→',f1_score(test_labels, y_pred))
print('precision→',precision_score(test_labels, y_pred))
print('recall→',recall_score(test_labels, y_pred))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6042262	test: 0.6257417	best: 0.6257417 (0)	total: 964ms	remaining: 1h 4m 13s
1:	learn: 0.5239883	test: 0.5639610	best: 0.5639610 (1)	total: 1.91s	remaining: 1h 3m 41s
2:	learn: 0.4627515	test: 0.5126662	best: 0.5126662 (2)	total: 2.9s	remaining: 1h 4m 20s
3:	learn: 0.4310122	test: 0.4855437	best: 0.4855437 (3)	total: 3.87s	remaining: 1h 4m 27s
4:	learn: 0.4045974	test: 0.4614312	best: 0.4614312 (4)	total: 4.83s	remaining: 1h 4m 21s
5:	learn: 0.3617144	test: 0.4252492	best: 0.4252492 (5)	total: 5.77s	remaining: 1h 4m 1s
6:	learn: 0.3353870	test: 0.4007025	best: 0.4007025 (6)	total: 6.7s	remaining: 1h 3m 40s
7:	learn: 0.3220227	test: 0.3880659	best: 0.3880659 (7)	total: 7.64s	remaining: 1h 3m 35s
8:	learn: 0.3009401	test: 0.3688628	best: 0.3688628 (8)	total: 8.59s	remaining: 1h 3m 29s
9:	learn: 0.2890659	test: 0.3573695	best: 0.3573695 (9)	total: 9.57s	remaining: 1h 3m 36s
10:	learn: 0.2804727	test: 0.3502889	best: 0.3502889 (10)	total: 10.5s	remaining: 1h 3m 44s
11:	learn: 